# Breakdown overview
1. subject: ESM_inverse_fold (https://github.com/facebookresearch/esm/tree/main/esm/inverse_folding)
2. aim of ESM_inverse_fold: conditon on backbone structure to recover sequence (software with similar functionality: ProteinMPNN)  
3. Overall architecture:  
Geometric GVP-GNN as initial layers, followed by
sequence-to-sequence Transformer encoder and decoder


## Geometric GVP_GNN layer (representation layer)
1. feature.py (take cooridnates to caluclate features (VECOTR OR SCALER))    
1.1 _diheadral (omega, psi, phi/ cos/sin  scaler)  
1.2 _orientation (CA-CA(i-1/i+1),  Vector)   
1.3 _sidechain  (calcualte CA-CB vector based on CA N CO, vector)  
1.4 _dist (calculate pair distance ), output k-neasrt neighbour ( dist (B,l,k),   E_idx (B,L,K), cooridnate_mask (B,L,K, whether the distance is calculable,such as   masked), residue_mask(B,L,K, whehter it is padding))      
1.5 _position_embedding: sinusoidual postion embedding  
1.6 get_node_feature: combine 1.1 and 1.2 1.3   
1.7 get_edge_feature: basically combine 1.4 and 1.5 (details are bit complex and   illustrated later) 

1.8 GVPGraphEmbedding: call 1.6 and 1.7 to get feature, use a GVP layer to embed these features 

2. gvp_module.py
2.1 basic model to handle mixture of vector scaler info and update embedding by passing info betwen vector and scaler   
vector's norm is concat to scaler to pass info  
scaler is used as gating to pass info  
![GVP](GVP_cartoon.png)

2.2 Message passing with GVP (GVP-GNN)
![GVP_GNN_1](GVP_GNN_1.png)
![GVP_GNN_2](GVP_GNN_2.png)

## Discussion of Invariant and equvarient for translation and rotation
1. translation
It is obvious that the model is invariant to translation as inter-atom vector and torsion angle were used for input. 
2. rotation 
GVP is only equivariant to ratation, i.e.  GVP(R.X)=R.GVP(X)
as Linear(Linear(R(x1, x2, x3)))= R(Linear(Linear(x1, x2, x3))), here Linear is a elementwise operation on indivually x1, x2, x3
GVP-GNN is invariant to ratation, as the GNN contain rotation invariant part.

## Transformer Encoder and Decoder layer

### Encoder layer:
x <- LayerNorm(x)  
x <- SelfAttention(x)    
x <- Dropout(x)    
x <- Residual_connect(x)  
x <- LayerNorm(x)    
x <- MLP(ReLu(MLP(x)))  
x <- Dropout(x)   
x <- Residual_connect(x)  

### Decoder layer:  

x <- LayerNorm(x)   
x <- SelfAttention(x) , with cached K V to avoid recomputate (explain: https://medium.com/@joaolages/kv-caching-explained-276520203249)     
x <- Dropout(x)    
x <- Residual_connect(x)  
x <- LayerNorm(x)    
x, attn <- Encoder Attention (q=x, key=encoder_output, v=encoder_output), cached KV    
x <- Dropout(x)   
x <- Residual_connect(x)  
x <- LayerNorm(x)   
x <- ReLu(MLP(x))  
x <- LayerNorm(x)   
x <- Dropout(MLP(x))   
x <- scaling(x) with learnable scaling factors  
x <- Residual_connect(x)  
output: x, attn  

## Build GVPTransfomer_encoder

the major part is to generate embedding x for transformer input
It contains:
1. initial features 
self.embed_gvp_input_features = nn.Linear(15, embed_dim)
scalar_features, vector_features = GVPInputFeaturizer.get_node_features(
            coords, coord_mask, with_coord_mask=False)
        features = torch.cat([
            scalar_features,
            rotate(vector_features, R.transpose(-2, -1)).flatten(-2, -1),
        ], dim=-1)
components["gvp_input_features"] = self.embed_gvp_input_features(features)
2. confidence embedding: rbf (bin=16)
self.embed_confidence = nn.Linear(16, embed_dim)
3. Dihedral angle (it seems redundant with scaler part in 1)
self.embed_dihedrals = DihedralFeatures(embed_dim)
4. output from GVP encoder 
gvp_out_dim = gvp_args.node_hidden_dim_scalar + (3 *
                gvp_args.node_hidden_dim_vector)
self.embed_gvp_output = nn.Linear(gvp_out_dim, embed_dim)
gvp_out_scalars, gvp_out_vectors = self.gvp_encoder(coords,
                coord_mask, padding_mask, confidence)
        R = get_rotation_frames(coords)
        # Rotate to local rotation frame for rotation-invariance
        gvp_out_features = torch.cat([
            gvp_out_scalars,
            rotate(gvp_out_vectors, R.transpose(-2, -1)).flatten(-2, -1),
        ], dim=-1)
components["gvp_out"] = self.embed_gvp_output(gvp_out_features)
5. mask token embedding 

mask_tokens = (
    padding_mask * self.dictionary.padding_idx + 
    ~padding_mask * self.dictionary.get_idx("<mask>")
)
components["tokens"] = self.embed_tokens(mask_tokens) * self.embed_scale


6. sinusoidual position embedding

x is sum over 1 to 6:
embed = sum(components.values())




## Build Transformer Decoder

###input x:

x = self.embed_scale * self.embed_tokens(prev_output_tokens)

if self.project_in_dim is not None:
    x = self.project_in_dim(x)

x += positions

###Decoder layer
enc: output from above GVPTransformerEncoder 

for idx, layer in enumerate(self.layers):
    if incremental_state is None:
        self_attn_mask = self.buffered_future_mask(x)
    else:
        self_attn_mask = None

    x, layer_attn, _ = layer(
        x,
        enc,
        padding_mask,
        incremental_state,
        self_attn_mask=self_attn_mask,
        self_attn_padding_mask=self_attn_padding_mask,
        need_attn=False,
        need_head_weights=False,
    )
    inner_states.append(x)

#Sampling 

There are a few strategies:

In [2]:
# greedy search
def greedy_search_algorithm(input_ids, model, max_length):
    for _ in range(max_length - input_ids.shape[1]):
        outputs = model(input_ids)
        logits = outputs.logits[:, -1, :]  # Get logits of the last token
        next_token = torch.argmax(logits, dim=-1)  # Select token with the highest probability
        input_ids = torch.cat([input_ids, next_token.unsqueeze(-1)], dim=-1)
        if next_token.item() == tokenizer.eos_token_id:  # Stop if EOS token is generated
            break
    return tokenizer.decode(input_ids[0], skip_special_tokens=True)

#multinomial sampling
def multinomial_sampling_algorithm(input_ids, model, max_length, temperature=1.0):
    for _ in range(max_length - input_ids.shape[1]):
        outputs = model(input_ids)
        logits = outputs.logits[:, -1, :]  # Get logits of the last token
        logits = logits / temperature  # Apply temperature scaling
        probabilities = torch.softmax(logits, dim=-1)
        next_token = torch.multinomial(probabilities, num_samples=1)  # Sample from the distribution
        input_ids = torch.cat([input_ids, next_token], dim=-1)
        if next_token.item() == tokenizer.eos_token_id:  # Stop if EOS token is generated
            break
    return tokenizer.decode(input_ids[0], skip_special_tokens=True)

#beam search

def beam_search_algorithm(input_ids, model, max_length, num_beams=5):
    sequences = [[input_ids, 0]]  # List of (sequence, score)
    for _ in range(max_length - input_ids.shape[1]):
        all_candidates = []
        for seq, score in sequences:
            outputs = model(seq)
            logits = outputs.logits[:, -1, :]
            probabilities = torch.softmax(logits, dim=-1)
            top_tokens = torch.topk(probabilities, num_beams, dim=-1)
            for token, prob in zip(top_tokens.indices[0], top_tokens.values[0]):
                candidate = [torch.cat([seq, token.unsqueeze(-1)], dim=-1), score - torch.log(prob)]
                all_candidates.append(candidate)
        sequences = sorted(all_candidates, key=lambda x: x[1])[:num_beams]  # Keep top beams
        if all(seq[0][-1].item() == tokenizer.eos_token_id for seq in sequences):
            break
    return tokenizer.decode(sequences[0][0][0], skip_special_tokens=True)

#beam search with multinomidal sampling

def beam_search_multinomial_algorithm(input_ids, model, max_length, num_beams=5, top_k=50):
    sequences = [[input_ids, 0]]  # List of (sequence, score)
    for _ in range(max_length - input_ids.shape[1]):
        all_candidates = []
        for seq, score in sequences:
            outputs = model(seq)
            logits = outputs.logits[:, -1, :]
            probabilities = torch.softmax(logits, dim=-1)
            top_k_tokens = torch.topk(probabilities, top_k, dim=-1)
            sampled_token = torch.multinomial(top_k_tokens.values[0], num_samples=1)
            token = top_k_tokens.indices[0, sampled_token]
            prob = top_k_tokens.values[0, sampled_token]
            candidate = [torch.cat([seq, token.unsqueeze(-1)], dim=-1), score - torch.log(prob)]
            all_candidates.append(candidate)
        sequences = sorted(all_candidates, key=lambda x: x[1])[:num_beams]
        if all(seq[0][-1].item() == tokenizer.eos_token_id for seq in sequences):
            break
    return tokenizer.decode(sequences[0][0][0], skip_special_tokens=True)

#contrasive search
def contrastive_search_algorithm(input_ids, model, max_length, top_k=5, penalty_alpha=0.6):
    generated = input_ids
    for _ in range(max_length - input_ids.shape[1]):
        outputs = model(generated)
        logits = outputs.logits[:, -1, :]
        top_k_logits, top_k_indices = torch.topk(logits, top_k, dim=-1)
        probabilities = torch.softmax(top_k_logits, dim=-1)
        diversity_penalty = penalty_alpha * torch.log(probabilities)
        adjusted_scores = probabilities - diversity_penalty
        next_token = top_k_indices[0, torch.argmax(adjusted_scores)]
        generated = torch.cat([generated, next_token.unsqueeze(-1)], dim=-1)
        if next_token.item() == tokenizer.eos_token_id:  # Stop if EOS token is generated
            break
    return tokenizer.decode(generated[0], skip_special_tokens=True)


In [3]:
#Sampling in ESM_IF is multinomial sampling
def sample(self, coords, partial_seq=None, temperature=1.0, confidence=None, device=None):
        """
        Samples sequences based on multinomial sampling (no beam search).

        Args:
            coords: L x 3 x 3 list representing one backbone
            partial_seq: Optional, partial sequence with mask tokens if part of
                the sequence is known
            temperature: sampling temperature, use low temperature for higher
                sequence recovery and high temperature for higher diversity
            confidence: optional length L list of confidence scores for coordinates
        """
        L = len(coords)
        # Convert to batch format
        batch_converter = CoordBatchConverter(self.decoder.dictionary)
        batch_coords, confidence, _, _, padding_mask = (
            batch_converter([(coords, confidence, None)], device=device)
        )
        
        # Start with prepend token
        mask_idx = self.decoder.dictionary.get_idx('<mask>')
        sampled_tokens = torch.full((1, 1+L), mask_idx, dtype=int)
        sampled_tokens[0, 0] = self.decoder.dictionary.get_idx('<cath>')
        if partial_seq is not None:
            for i, c in enumerate(partial_seq):
                sampled_tokens[0, i+1] = self.decoder.dictionary.get_idx(c)
            
        # Save incremental states for faster sampling
        incremental_state = dict()
        
        # Run encoder only once
        encoder_out = self.encoder(batch_coords, padding_mask, confidence)
        
        # Make sure all tensors are on the same device if a GPU is present
        if device:
            sampled_tokens = sampled_tokens.to(device)
        
        # Decode one token at a time
        for i in range(1, L+1):
            logits, _ = self.decoder(
                sampled_tokens[:, :i], 
                encoder_out,
                incremental_state=incremental_state,
            )
            logits = logits[0].transpose(0, 1)
            logits /= temperature
            probs = F.softmax(logits, dim=-1)
            if sampled_tokens[0, i] == mask_idx:
                sampled_tokens[:, i] = torch.multinomial(probs, 1).squeeze(-1)
        sampled_seq = sampled_tokens[0, 1:]
        
        # Convert back to string via lookup
        return ''.join([self.decoder.dictionary.get_tok(a) for a in sampled_seq])